In [17]:
#importing necessary libraries
import opendatasets as od
import pandas as pd
import pymongo
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
#reading csv files
client=pymongo.MongoClient("mongodb://127.0.0.1:27017/")
mydb=client['Mars']
info=mydb.SurfaceAnalysis
cursor = info.find()
entries=list(cursor)
mars = pd.DataFrame(entries)
mars2 = pd.DataFrame(entries)
mars3 = pd.DataFrame(entries)

mars.head()
mars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50901 entries, 0 to 50900
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   _id                   50901 non-null  object
 1   earth_date            3197 non-null   object
 2   mars_date_time        3197 non-null   object
 3   sol                   3197 non-null   object
 4   max_ground_temp       3169 non-null   object
 5   min_ground_temp       3169 non-null   object
 6   max_air_temp          3168 non-null   object
 7   min_air_temp          3168 non-null   object
 8   mean_pressure         3170 non-null   object
 9   humidity              3197 non-null   object
 10  sunrise               3197 non-null   object
 11  sunset                3197 non-null   object
 12  uv_radiation          3170 non-null   object
 13  weather               3197 non-null   object
 14  Latitude °            50901 non-null  object
 15  Latitude              50901 non-null

In [18]:
mars=mars.rename(columns={'earth_date_time': 'earth_date',
                     'sol_number':'sol',
                     'max_ground_temp(°C)':'max_ground_temp',
                     'min_ground_temp(°C)':'min_ground_temp',
                     'max_air_temp(°C)':'max_air_temp',
                     'min_air_temp(°C)':'min_air_temp',
                     'mean_pressure(Pa)':'mean_pressure',
                     'humidity(%)':'humidity',
                     'UV_Radiation':'uv_radiation'} )

mars["sol"]=mars.sol.str.replace("Sol ,?","")
mars["earth_date"]=mars.earth_date.str.replace("Earth, | UTC,?","")
mars["mars_date_time"]=mars.mars_date_time.str.replace("°,?","")

#force sol to be an numeric integer
mars["sol"] = pd.to_numeric(mars["sol"], errors='coerce')

#convert sol to be int
mars["sol"] = mars["sol"].astype("int", errors='ignore')
# mars["sunrise"]= pd.to_datetime(mars["sunrise"])
# mars['sunrise'] = pd.Series([val.time() for val in mars['sunrise']])
# mars["sunset"]= pd.to_datetime(mars["sunset"])
# mars['sunset'] = pd.Series([val.time() for val in mars['sunset']])


# mars['sunrise'] = pd.to_datetime(mars['sunrise'], format='%H:%M')


# mars['sunrise'] = mars['sunrise'].apply(pd.Timestamp)
# mars['sunrise'].apply(pd.Timestamp)

# mars['sunset'] = pd.to_datetime(mars['sunset'], format='%H:%M')
# mars['sunset'] = mars['sunset'].apply(pd.Timestamp)
# mars['sunset'].apply(pd.Timestamp)

mars["max_ground_temp"] = mars["max_ground_temp"].astype("int", errors='ignore')
mars["max_ground_temp"] = pd.to_numeric(mars["max_ground_temp"], errors='coerce')
#converting max_ground_temp to numeric
mars["min_ground_temp"] = mars["min_ground_temp"].astype("int", errors='ignore')
#convrting min_ground_temp to numeric!
mars["min_ground_temp"] = pd.to_numeric(mars["min_ground_temp"], errors='coerce')
mars["max_air_temp"] = mars["max_air_temp"].astype("int", errors='ignore')
#converting max_air_temp to numeric
mars["max_air_temp"] = pd.to_numeric(mars["max_air_temp"], errors='coerce')
# adding Min_Air_Temp column
mars["min_air_temp"] = mars["min_air_temp"].astype("int", errors='ignore')
#converting min_air_temp to numeric
mars["min_air_temp"] = pd.to_numeric(mars["min_air_temp"], errors='coerce')
mars["mean_pressure"] = mars["mean_pressure"].astype("int", errors='ignore')
#converting mean pressure to numeric
mars["mean_pressure"] = pd.to_numeric(mars["mean_pressure"], errors='coerce')



mars['avg_ground_temp'] = mars[['max_ground_temp', 'min_ground_temp']].mean(axis=1)
mars['avg_air_temp'] = mars[['max_air_temp', 'min_air_temp']].mean(axis=1)



mars

/var/folders/_2/bwnpndcd41l350rv63l_1rk80000gn/T/ipykernel_4482/2518278527.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  mars["sol"]=mars.sol.str.replace("Sol ,?","")
/var/folders/_2/bwnpndcd41l350rv63l_1rk80000gn/T/ipykernel_4482/2518278527.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  mars["earth_date"]=mars.earth_date.str.replace("Earth, | UTC,?","")
/var/folders/_2/bwnpndcd41l350rv63l_1rk80000gn/T/ipykernel_4482/2518278527.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  mars["mars_date_time"]=mars.mars_date_time.str.replace("°,?","")


,_id,earth_date,mars_date_time,sol,max_ground_temp,min_ground_temp,max_air_temp,min_air_temp,mean_pressure,humidity,...,Concrete,Water,Dust Devils,Dust Storms,Meteors,Cold Waves,Map Name,avg_ground_temp,avg_air_temp,Named Location
0,63921cff5c7c7bce96da1cb4,2022-01-26,"Mars, Month 6 - LS 163",3368.0,-3.0,-71.0,10.0,-84.0,707.0,Value not available,...,4,1,2,1,4,1,BlankBigCliffsCMix_02,-37.0,-37.0,NaN
1,63921cff5c7c7bce96da1cb5,2022-01-25,"Mars, Month 6 - LS 163",3367.0,-3.0,-72.0,10.0,-87.0,707.0,Value not available,...,4,1,2,1,4,1,BlankBig_03,-37.5,-38.5,NaN
2,63921cff5c7c7bce96da1cb6,2022-01-24,"Mars, Month 6 - LS 162",3366.0,-4.0,-70.0,8.0,-81.0,708.0,Value not available,...,4,1,2,2,4,1,BlankBig_01,-37.0,-36.5,NaN
3,63921cff5c7c7bce96da1cb7,2022-01-23,"Mars, Month 6 - LS 162",3365.0,-6.0,-70.0,9.0,-91.0,707.0,Value not available,...,4,1,2,1,4,1,BlankBigTerraceCMix_16,-38.0,-41.0,NaN
4,63921cff5c7c7bce96da1cb8,2022-01-22,"Mars, Month 6 - LS 161",3364.0,-7.0,-71.0,8.0,-92.0,708.0,Value not available,...,4,1,2,3,4,1,BlankBig_01,-39.0,-42.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50896,63921d0b5c7c7bce96dae384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,1,1,2,1,4,BlankBig_02,NaN,NaN,NaN
50897,63921d0b5c7c7bce96dae385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,2,1,1,4,BlankBigTerraceCMix_10,NaN,NaN,NaN
50898,63921d0b5c7c7bce96dae386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,2,1,1,4,BlankBigTerraceCMix_11,NaN,NaN,NaN
50899,63921d0b5c7c7bce96dae387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,1,1,2,1,4,BlankBigTerraceCMix_06,NaN,NaN,NaN


In [19]:
mars.columns

Index(['_id', 'earth_date', 'mars_date_time', 'sol', 'max_ground_temp',
       'min_ground_temp', 'max_air_temp', 'min_air_temp', 'mean_pressure',
       'humidity', 'sunrise', 'sunset', 'uv_radiation', 'weather',
       'Latitude °', 'Latitude', 'Longitude °', 'Longitude', 'Topography',
       'Difficulty Challenge', 'Altitude', 'Temperature', 'Metals',
       'Rare Metals', 'Concrete', 'Water', 'Dust Devils', 'Dust Storms',
       'Meteors', 'Cold Waves', 'Map Name', 'avg_ground_temp', 'avg_air_temp',
       'Named Location'],
      dtype='object')

In [21]:
#creating solar long
# 
# #converting solar_long  to a datetime format and removing string artifacts
# with regex
mars['solar_long'] = mars['mars_date_time'].str.replace(r'.*LS', '')

#convert solar_long to numeric
mars['solar_long'] = pd.to_numeric(mars['solar_long'], errors='coerce')




mars['sunrise'] = pd.to_datetime(mars['sunrise'], format='%H:%M')
mars['sunset'] = pd.to_datetime(mars['sunset'], format='%H:%M')




#Creating conditions on which map season to the condition for future analysis

conditions = [
    (mars["solar_long"] >= 0) & (mars["solar_long"] < 90),
    (mars["solar_long"] >= 90) & (mars["solar_long"] < 180),
    (mars["solar_long"] >= 180) & (mars["solar_long"] < 270),
    (mars["solar_long"] >= 270) & (mars["solar_long"] <= 360)
]
values = ['Spring', 'Summer', 'Autumn', 'Winter']

mars['season'] = np.select(conditions, values)

#Creating conditions which map days to the condition of years for future analysis


conditions = [
    (mars["sol"] >= 0) & (mars["sol"] < 351),
    (mars["sol"] >= 351) & (mars["sol"] < 1019),
    (mars["sol"] >= 1019) & (mars["sol"] < 1688),
    (mars["sol"] >= 1688) & (mars["sol"] <= 2357),
    (mars["sol"] >= 2357) & (mars["sol"] <= 3024),
    (mars["sol"] >= 3024) & (mars["sol"] <= 3368)
]

values = ['Year 31', 'Year 32', 'Year 33', 'Year 34','Year 35','Year 36']

mars['mars_years'] = np.select(conditions, values)



# create a condition, only use yeears that had less redundant data.
options = ['Year 32', 'Year 33','Year 34','Year 35'] 
    
# # selecting rows based on condition 
mars = mars[mars['mars_years'].isin(options)]



# # mars["duration"] = datetime.combine(date.min, end) - datetime.combine(date.min, beginning)
mars['day_duration_in_mins'] = mars['sunset'] - mars['sunrise']
mars['day_duration_in_mins']=mars['day_duration_in_mins']/np.timedelta64(1,'m')




mars.head()

/var/folders/_2/bwnpndcd41l350rv63l_1rk80000gn/T/ipykernel_4482/4273496778.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  mars['solar_long'] = mars['mars_date_time'].str.replace(r'.*LS', '')
/var/folders/_2/bwnpndcd41l350rv63l_1rk80000gn/T/ipykernel_4482/4273496778.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mars['solar_long'] = mars['mars_date_time'].str.replace(r'.*LS', '')
/var/folders/_2/bwnpndcd41l350rv63l_1rk80000gn/T/ipykernel_4482/4273496778.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

ValueError: time data '1900-01-01 06:41:00' does not match format '%H:%M' (match)